In [3]:
def tokenize(element):
    messages = [
        {"role": "user", "content": element["prompt"]}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False # Switches between thinking and non-thinking modes. Default is True.
    )
    print(text)
    input_ids = tokenizer.encode(
        text,
        padding=False,
    )
    text = element["response"] + tokenizer.eos_token
    print(text)
    response_ids = tokenizer.encode(
        text,
        padding=False,
    )
    reward = (element["reward"])
    return {"input_ids": input_ids, "response_ids": response_ids, "reward": reward, "lengths": len(input_ids)}

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/root/group-shared/jrc/base-models/Qwen3-32B")

element = dict(
    prompt = "你好",
    response = "我也好",
    reward = 1
)

tokenize(element)

<|im_start|>user
你好<|im_end|>
<|im_start|>assistant
<think>

</think>


我也好<|im_end|>


{'input_ids': [151644,
  872,
  198,
  108386,
  151645,
  198,
  151644,
  77091,
  198,
  151667,
  271,
  151668,
  271],
 'response_ids': [104284, 52801, 151645],
 'reward': 1,
 'lengths': 13}

In [4]:
tokenize(element)

<|im_start|>user
你好<|im_end|>
<|im_start|>assistant
<think>

</think>


我也好<|im_end|>


{'input_ids': [151644,
  872,
  198,
  108386,
  151645,
  198,
  151644,
  77091,
  198,
  151667,
  271,
  151668,
  271],
 'response_ids': [104284, 52801, 151645],
 'reward': 1,
 'lengths': 13}

In [9]:

print(tokenizer.decode([
        151644,
        872,
        198,
        108386,
        151645,
        198,
        151644,
        77091,
        198,
        151667,
        271,
        151668,
        271,
        104284,
        52801,
        151645
    ])
)

<|im_start|>user
你好<|im_end|>
<|im_start|>assistant
<think>

</think>

我也好<|im_end|>


In [10]:
from transformers import AutoModelForCausalLM
model_name = "/root/group-shared/models/base_models/Qwen3-32B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    attn_implementation="flash_attention_2",
)

Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

In [12]:
messages = [
    {"role": "user", "content": "你好"}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

In [14]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768,
    temperature=1.0
)
output_ids = generated_ids[0].tolist() 
print(tokenizer.decode(output_ids, skip_special_tokens=False))

<|im_start|>user
你好<|im_end|>
<|im_start|>assistant
<think>

</think>

你好！很高兴见到你✨ 今天过得怎么样呀？希望你度过了愉快的一天。我随时准备好陪你聊天、帮你解决问题，或者就这样轻松愉快地闲聊一会儿。有什么想跟我分享的吗？ 🌟<|im_end|>
